In [5]:
from abc import ABC
from jinja2 import Template
from datetime import datetime
import uuid

from jinja2 import UndefinedError
import logging 
import copy

logging.basicConfig(format='%(asctime)s %(message)s',
                    filemode='w')
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

days_of_the_week = {
    0: "Maandag",
    1: "Dinsdag",
    2: "Woensdag",
    3: "Donderdag",
    4: "Vrijdag",
    5: "Zaterdag",
    6: "Zondag"
}

months = {
    1: "Januari",
    2: "Februari",
    3: "Maart",
    4: "April",
    5: "Mei",
    6: "Juni",
    7: "Juli",
    8: "Augustus",
    9: "September",
    10: "Oktober",
    11: "November",
    12: "December",
}

class LatexSyntax:
    OPEN = '{'
    CLOSE = '}'
    BACKSLASH = chr(92)
    PERCENTAGE = "%"

class LatexCommands:
    BEGINFLUSHRIGHT = str(Template(
        """{{ latex.BACKSLASH }}begin{{ latex.OPEN }}flushright{{ latex.CLOSE }}""").render(latex=LatexSyntax()))
    ENDFLUSHRIGHT = str(Template(
        """{{ latex.BACKSLASH }}end{{ latex.OPEN }}flushright{{ latex.CLOSE }}""").render(latex=LatexSyntax()))
    BEGINCOLORBOX = str(Template(
        """{{ latex.BACKSLASH }}begin{{ latex.OPEN}}tcolorbox{{ latex.CLOSE }}""").render(latex=LatexSyntax()))
    ENDCOLORBOX = str(Template(
        """{{ latex.BACKSLASH }}end{{ latex.OPEN}}tcolorbox{{ latex.CLOSE }}""").render(latex=LatexSyntax()))

class LatexTemplates:
    DAY_SEGMENT = Template("""
    {{ latex.BACKSLASH }}begin{{ latex.OPEN }}center{{ latex.CLOSE }}
    {{ latex.BACKSLASH }}begin{{ latex.OPEN }}tcolorbox{{ latex.CLOSE }}[
        colback=whatsapplightblue, 
        colframe=whatsapplightblue, 
        width=3.0cm,
        arc=3mm, auto outer arc,
        halign=center,
        valign=center,
        ]
    {{ latex.BACKSLASH }}scriptsize  {{ message.metadata.day }} {{ message.metadata.date }}
    {{ latex.BACKSLASH }}end{{ latex.OPEN }}tcolorbox{{ latex.CLOSE }}
    {{ latex.BACKSLASH }}end{{ latex.OPEN }}center{{ latex.CLOSE }}
    """)

    MONTH_SEGMENT = Template("""
    {{ latex.BACKSLASH }}begin{{ latex.OPEN }}center{{ latex.CLOSE }}
    {{ latex.BACKSLASH }}begin{{ latex.OPEN }}tcolorbox{{ latex.CLOSE }}[
        colback=whatsapplightblue, 
        colframe=whatsapplightblue, 
        width=3.0cm,
        arc=3mm, auto outer arc,
        halign=center,
        valign=center,
        ]
    {{ latex.BACKSLASH }}normalsize  {{ message.metadata.month }}
    {{ latex.BACKSLASH }}end{{ latex.OPEN }}tcolorbox{{ latex.CLOSE }}
    {{ latex.BACKSLASH }}end{{ latex.OPEN }}center{{ latex.CLOSE }}
    """)

    NORMAL = Template("""      
    {% if message.metadata.person.perspective == True -%}
    {{ command.BEGINFLUSHRIGHT }}
    {% endif -%}
    {{ command.BEGINCOLORBOX }}[
        colback={{ message.metadata.colour }}, 
        colframe={{ message.metadata.colour }}, 
        width={{ message.metadata.width }}cm,
        arc=3mm, auto outer arc,
        halign=left,
        {% if message.metadata.person.name != message.previous_message_metadata.person.name -%}
        title = {{ latex.BACKSLASH }}color{{ latex.OPEN}}{{message.metadata.person.colour }}{{ latex.CLOSE }} {{ message.metadata.person.name }},
        {% endif -%}
        ]
    {{ latex.BACKSLASH}}footnotesize {{ message.text }}
        {{ command.BEGINFLUSHRIGHT }}
            {{ latex.BACKSLASH }}tiny {{ latex.BACKSLASH }}textbf{{ latex.OPEN }}{{ message.metadata.time}}{{ latex.CLOSE }} {{ latex.BACKSLASH }}par  
        {{ command.ENDFLUSHRIGHT }}
    {{ command.ENDCOLORBOX }}
    {% if message.metadata.person.perspective == True -%}
    {{ command.ENDFLUSHRIGHT }}
    {% endif -%}
        """)

    IMAGE = Template("""      
    {% if message.metadata.person.perspective == True -%}
    {{ command.BEGINFLUSHRIGHT }}
    {% endif -%}
    {{ command.BEGINCOLORBOX }}[
        colback={{ message.metadata.colour }}, 
        colframe={{ message.metadata.colour }}, 
        width=6.75cm,
        arc=3mm, auto outer arc,
        halign=left,
        {% if message.metadata.person.name != message.previous_message_metadata.person.name -%}
        title = {{ latex.BACKSLASH }}color{{ latex.OPEN}}{{message.metadata.person.colour }}{{ latex.CLOSE }} {{ message.metadata.person.name }},
        {% endif -%}
        ]

    {% if message.metadata.person.perspective == True -%}
    {{ command.BEGINFLUSHRIGHT }}
    {% endif -%}
        {{ latex.BACKSLASH }}begin{figure}[H]
        {{ latex.BACKSLASH }}includegraphics[width=5.75cm, right]{{ latex.OPEN}}{{ message.metadata.path}}{{ latex.CLOSE}}
        {{ latex.BACKSLASH }}end{figure}   
    {% if message.metadata.person.perspective == True -%}
    {{ command.ENDFLUSHRIGHT }}
    {% endif -%}    
    {{ command.BEGINFLUSHRIGHT }}
            {{ latex.BACKSLASH }}tiny {{ latex.BACKSLASH }}textbf{{ latex.OPEN }}{{ message.metadata.time}}{{ latex.CLOSE }} {{ latex.BACKSLASH }}par  
        {{ command.ENDFLUSHRIGHT }}
    {{ command.ENDCOLORBOX }}
    {% if message.metadata.person.perspective == True -%}
    {{ command.ENDFLUSHRIGHT}}
    {% endif -%}
        """)


class Person:
    def __init__(self, contact_name, name, colour, id, perspective=False):
        self = self
        self.contact_name = contact_name
        self.name = name
        self.colour = colour
        self.perspective = perspective
        self.id = id


class MessageMetaData:
    def __init(self):
        self.id = None
        self.message_type = None
        self.date = None
        self.time = None
        self.month = None
        self.day_of_week = None
        self.day = None
        self.name = None
        self.person = None
        self.colour = None
        self.width = None
        self.attachment_type = None
        self.first_message = None
        self.new_date = None
        self.new_month = None
        
    def get_metadata(self):
        return self


class Message(ABC):
    def __init__(self, message, id, persons, message_type, metadata, previous_message_metadata, group, current_date,current_month):
        self = self
        self.message = message
        self.persons = persons
        self.message_type = message_type
        self.metadata = metadata
        self.previous_message_metadata = previous_message_metadata
        self.group = group
        self.current_date = current_date
        self.current_month = current_month

    def exception_handler(self):
        if self.message == '[27-02-2022 12:32:28] ‎ +31617146903 Floris van gelder heeft de groep Floris & Tamara, LMS 56 gemaakt':
            raise Exception("Floris Jan")
        return self

    def get_date(self):
        self.date_start = self.message.find('[') + 1
        self.date_end = self.message.find(']')
        date_segment = self.message[self.date_start:self.date_end]
        self.metadata.date, self.metadata.time = date_segment.split(' ', 1)
        self.metadata.time = self.metadata.time[0:5]
        try:
            self.metadata.day_of_week = datetime.strptime(
                self.metadata.date, "%d-%m-%Y").weekday()
            self.metadata.month = datetime.strptime(
                self.metadata.date, "%d-%m-%Y").month
            self.metadata.month = months.get(self.metadata.month)
            self.metadata.day = days_of_the_week.get(self.metadata.day_of_week)

            if self.current_date == self.metadata.date:
                self.metadata.new_date = False
            else:
                self.metadata.new_date = True        
          
        except IndexError as error:
            logger.error(error,self.metadata.date)
        except AttributeError as error:
            logger.error(error,self.metadata.date)
        return self

    def get_new_date(self):
        try:
            if self.current_date == self.metadata.date:
                self.metadata.new_date = False
            else:
                self.metadata.new_date = True        
        except IndexError as error:
            logger.error(error,self.metadata.date)
        except AttributeError as error:
            logger.error(error,self.metadata.date)

        try:
            if self.current_month == self.metadata.month:
                self.metadata.new_month = False
            else:
                self.metadata.new_month = True        
        except IndexError as error:
            logger.error(error,self.metadata.date)
        except AttributeError as error:
            logger.error(error,self.metadata.date)
        return self

    def get_name(self):
        self.name_start = self.date_end
        self.name_end = self.message[self.name_start:].find(':')
        self.metadata.name = self.message[self.name_start +
                                          2:(self.name_start + self.name_end)]
        return self

    def get_person(self):
        try:
            self.metadata.person = [
                person for person in self.persons if person.contact_name == self.metadata.name][0]
        except IndexError:
            self.metadata.person = Person(name="Idioot", contact_name="Idioot",
               colour="Red", id=999, perspective=False)
        return self

    def get_message_colour(self):
        if self.metadata.person.perspective == True:
            self.metadata.colour = "whatsappgreen"
        else:
            self.metadata.colour = "whatsappgray"
        return self

    def get_message_width(self):
        self.metadata.width = sorted([3, int(len(self.text)) * 0.1, 9.5])[1]
        return self

    def get_print_name(self):
        try:
            if self.group:
                if self.previous_message_metadata.person.id is None:
                    self.metadata.print_name = False
                elif self.previous_message_metadata.person.id == self.metadata.person.id:
                    self.metadata.print_name = False
                else:
                    self.metadata.print_name = True
            else:
                self.metadata.print_name = False
        except AttributeError as error:
            logger.error(f"{error}")

        return self


class NormalMessage(Message):
    def get_message(self):
        self.text = self.message[(
            self.name_start + self.name_end + self.date_start)::]
        return self


class MessageWithoutSenderMetaData(Message):
    def get_message(self):
        self.text = self.message
        return self


class AttachmentMessage(Message):
    def get_attachment(self):
        attachment_start = self.message.find('<') + 1
        attachment_end = self.message.find('>')
        self.attachment = self.message[attachment_start:attachment_end]
        self.attachment = self.attachment.split(':', 1)
        self.attachment = self.attachment[1][1::]
        return self

    def get_attachment_type(self):
        try:
            self.metadata.attachment_type = self.attachment.split('.', 1)
            self.metadata.attachment_type = self.metadata.attachment_type[1]
        except IndexError as error:
            logger.error({error})
            self.metadata.attachment_type = "unknown"
        return self

    def get_message_width(self):
        self.metadata.width = 6
        return self

    def get_attachment_path(self, folder):
        if self.metadata.attachment_type == 'jpg':
            self.metadata.path = "chats/" + folder + '/' + self.attachment 
        return self


class ChatHistory:
    def __init__(self, chat_name: str, persons: list,folder:str, group: bool):
        self = self
        self.name = chat_name
        self.persons = persons
        self.group = group
        self.folder = folder
        self.last_message_metadata = MessageMetaData()
        self.current_date = None
        self.current_month = None

    def open_chat(self, path):
        self.chat = open(path, "r")
        return self

    def read_history(self):
        self.chat_history = []
        id = 0
        
        for messages in self.chat:
            metadata = MessageMetaData()
            if id == 0:
                self.chat_history.append(f"Skip message {id}")
                id += 1
                continue
                
            if len(messages) == 0:
                self.chat_history.append(f"Skip message {id}")
                id += 1
                continue
            
            if self.group:
                if id <4:
                    id += 1
                    print(messages)
                    continue
                
            if messages[0] == '\u200e':
                MessageWithAttachment = AttachmentMessage(message=messages, id=id, persons=self.persons, message_type='attachment',
                                                          metadata=metadata, previous_message_metadata=self.last_message_metadata, group=self.group, current_date = self.current_date, current_month = self.current_month)
                (
                    MessageWithAttachment
                    .exception_handler()
                    .get_date()
                    .get_new_date()
                    .get_name()
                    .get_attachment()
                    .get_attachment_type()
                    .get_person()
                    .get_attachment_path(folder = self.folder)
                    .get_message_colour()
                    .get_message_width()
                    .get_print_name()
                )
                self.chat_history.append(MessageWithAttachment)
                self.last_message_metadata = MessageWithAttachment.metadata
                self.current_date, self.current_month = MessageWithAttachment.metadata.date,MessageWithAttachment.metadata.month
                id += 1
            elif messages[0] != '[':
                MessageWithoutMetaData = MessageWithoutSenderMetaData(message=messages, id=id, persons=self.persons, message_type='no_sender', metadata=self.last_message_metadata.get_metadata(
                ), previous_message_metadata=self.last_message_metadata, group=self.group,current_date = self.current_date,current_month = self.current_month)
                (MessageWithoutMetaData
                 .exception_handler()
                 .get_message()
                 .get_message_width()
                 .get_print_name()
                 .get_new_date()
                 )
                self.chat_history.append(MessageWithoutMetaData)
                self.last_message_metadata = copy.deepcopy(MessageWithoutMetaData.metadata)
                self.current_date, self.current_month = MessageWithoutMetaData.metadata.date,MessageWithoutMetaData.metadata.month
                id += 1
            else:
                Message = NormalMessage(message=messages, id=id, persons=self.persons, message_type='normal',
                                        metadata=metadata, previous_message_metadata=self.last_message_metadata, group=self.group,current_date = self.current_date,current_month = self.current_month)
                (
                    Message
                    .exception_handler()
                    .get_date()
                    .get_new_date()
                    .get_name()
                    .get_message()
                    .get_person()
                    .get_message_colour()
                    .get_message_width()
                    .get_print_name()
                )
                self.chat_history.append(Message)
                self.last_message_metadata = copy.deepcopy(Message.metadata)
                self.current_date, self.current_month = Message.metadata.date,Message.metadata.month
                id += 1            
        return self


class LatexGenerator:
    def __init__(self, chat_history):
        self = self
        self.chat_history = chat_history

    def generate_latex(self):
        self.storage = []
        id = 0
        for message in self.chat_history.chat_history:
            if type(message) == str:
                continue
                
            try:
                if message.metadata.new_date:
                    if self.chat_history.chat_history[id-1].metadata.new_date:
                        continue
                    rendered_template = LatexTemplates.DAY_SEGMENT.render(
                        message=message,
                        latex=LatexSyntax(),
                        command=LatexCommands()
                    )
                    self.storage.append(rendered_template)

                if message.metadata.new_month:
                    if self.chat_history.chat_history[id-1].metadata.new_month:
                        continue
                    rendered_template = LatexTemplates.MONTH_SEGMENT.render(
                        message=message,
                        latex=LatexSyntax(),
                        command=LatexCommands()
                    )
                    self.storage.append(rendered_template)

                if message.message_type in ['normal', 'no_sender']:
                    if len(message.text) == 1:
                        continue
                    rendered_template = LatexTemplates.NORMAL.render(
                        message=message,
                        latex=LatexSyntax(),
                        command=LatexCommands()
                    )
                elif message.message_type == 'attachment':
                    if message.metadata.attachment_type == 'jpg':
                        rendered_template = LatexTemplates.IMAGE.render(
                            message=message,
                            latex=LatexSyntax(),
                            command=LatexCommands()
                        )
                else:
                    logger.error("Unknown behaviour")
                self.storage.append(rendered_template)
                id += 1
            except UndefinedError as error:
                logger.error(error) 
            except AttributeError as error:
                logger.error(error)
        return self


In [11]:
gerda = ChatHistory(
    chat_name="Chat Gerda",
    folder = "gerda",
    persons=[
        Person(name="Irma", contact_name="Irma",
               colour="RawSienna", id=1, perspective=False),
        Person(name="Gerda", contact_name="Gerda Versteegen",
               colour="NavyBlue", id=2, perspective=True)
    ], group=False
).open_chat(path="WhatsApp Chat - Gerda Versteegen/_chat.txt").read_history()

tamara = ChatHistory(
    chat_name="Chat Tamara",
    folder = "tamara",
    persons=[
        Person(name="Irma", contact_name="Irma",
               colour="RawSienna", id=1, perspective=False),
        Person(name="Tamara", contact_name="Tamara",
               colour="BurntOrange", id=2, perspective=True)
    ], group=False
).open_chat(path="WhatsApp Chat - Tamara/_chat.txt").read_history()

plony = ChatHistory(
    chat_name="Chat Plony",
    folder = "plony",
    persons=[
        Person(name="Irma", contact_name="Irma",
               colour="RawSienna", id=1, perspective=False),
        Person(name="Plony", contact_name="Plony Hendriks",
               colour="YellowGreen", id=2, perspective=True)
    ], group=False
).open_chat(path="WhatsApp Chat - Plony Hendriks/_chat.txt").read_history()

thea = ChatHistory(
    chat_name="Chat Thea",
    folder = "thea",
    persons=[
        Person(name="Irma", contact_name="Irma",
               colour="RawSienna", id=1, perspective=True),
        Person(name="Plony", contact_name="Plony Hendriks",
               colour="YellowGreen", id=2, perspective=False),
        Person(name="Tamara", contact_name="Tamara",
               colour="BurntOrange", id=3, perspective=False),
        Person(name="Gerda", contact_name="Gerda Versteegen",
               colour="NavyBlue", id=4, perspective=False),
        Person(name="Floris", contact_name="+31617146903 Floris van gelder",
               colour="Red", id=5, perspective=False),
       Person(name="Karin",contact_name="Karin Lommers",colour="Purple",id=6,perspective=False)
    ], group=True
).open_chat(path="WhatsApp Chat - Thea/_chat.txt").read_history()

generator = LatexGenerator(chat_history=gerda).generate_latex()

for test in generator.storage[0:100000]:
    print(test)


2023-02-10 21:08:51,127 {IndexError('list index out of range')}
2023-02-10 21:08:51,177 {IndexError('list index out of range')}
2023-02-10 21:08:51,342 {IndexError('list index out of range')}
2023-02-10 21:08:51,541 {IndexError('list index out of range')}
2023-02-10 21:08:51,637 'MessageMetaData' object has no attribute 'person'


[27-02-2022 12:32:28] ‎+31617146903 Floris van gelder heeft de groep Floris & Tamara, LMS 56 gemaakt

[27-02-2022 12:32:28] ‎+31617146903 Floris van gelder heeft u toegevoegd

[27-02-2022 14:16:43] Tamara: 🙋‍♀️



2023-02-10 21:08:52,246 'str' object has no attribute 'metadata'


      
    \begin{tcolorbox}[
        colback=whatsappgray, 
        colframe=whatsappgray, 
        width=6.300000000000001cm,
        arc=3mm, auto outer arc,
        halign=left,
        title = \color{RawSienna} Irma,
        ]
    \footnotesize  Dank je wel voor je mail en je telefoonnummers. Ik sla ze op.

        \begin{flushright}
            \tiny \textbf{22:08} \par  
        \end{flushright}
    \end{tcolorbox}
    
      
    \begin{flushright}
    \begin{tcolorbox}[
        colback=whatsappgreen, 
        colframe=whatsappgreen, 
        width=9.5cm,
        arc=3mm, auto outer arc,
        halign=left,
        title = \color{NavyBlue} Gerda,
        ]
    \footnotesize  Geen dank. Ik hoop dat je moeder weer snel fit is. Geef maar een seintje als het nog een keer nodig is

        \begin{flushright}
            \tiny \textbf{22:07} \par  
        \end{flushright}
    \end{tcolorbox}
    \end{flushright}
    
      
    \begin{tcolorbox}[
        colback=whatsappgray, 
    